# Final Project Code

### Bailey Blum, Molly Giglia, Horacio Lopez & Ale Lopez

# DONT RUN THIS --------------

In [1]:
import json
import bigjson
import ijson
import pandas as pd
    

with open('yelp_dataset/review.json') as json_file:      
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data)) 

In [2]:
texts = [d['text'] for d in data]
stars = [d['stars'] for d in data]

In [3]:
from collections import Counter
 
def balance_classes(texts, stars):

    freqs = Counter(stars)

    # the least common class is the maximum number we want for all classes
    max_allowable = 25000
    #max_allowable = freqs.most_common()[-1][1]
    num_added = {clss: 0 for clss in freqs.keys()}
    new_stars = []
    new_texts = []
    for i, y in enumerate(stars):
        if num_added[y] < max_allowable:
            new_stars.append(y)
            new_texts.append(texts[i])
            num_added[y] += 1
    return new_texts, new_stars

In [4]:
#print(Counter(stars))
balanced_x, balanced_y = balance_classes(texts, stars)
print(Counter(balanced_y))
#print(balanced_y[:10])

Counter({1.0: 25000, 5.0: 25000, 4.0: 25000, 3.0: 25000, 2.0: 25000})


In [5]:
import os
os.remove("texts.txt")
os.remove("stars.txt")
f = open("texts.txt", "w")
f2 = open("stars.txt", "w")
for item in balanced_x:
    f.write(item + "Gigliablurb")
for item in balanced_y:
    f2.write(str(int(item)) + '\n')
f.close()
f2.close()

# RUN FROM HERE DOWN -------------------

In [1]:
import json
import bigjson
import ijson
import pandas as pd
from collections import Counter

f = open("texts.txt", "r")
f2 = open("stars.txt", "r")

balanced_x = []
balanced_y = []
listOfLines = f.read().split('Gigliablurb')
for item in listOfLines:
    balanced_x.append(item)
for item in f2.readlines():
    balanced_y.append(item)
    
f.close()
f2.close()

balanced_x = balanced_x[:-1]
print(len(balanced_x))

125000


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime
 
# This vectorizer breaks text into single words and bi-grams
# and then calculates the TF-IDF representation
vectorizer = TfidfVectorizer(ngram_range=(1,2))
t1 = datetime.datetime.now()
 
# the 'fit' builds up the vocabulary from all the reviews
# while the 'transform' step turns each indivdual text into
# a matrix of numbers.
vectors = vectorizer.fit_transform(balanced_x)
print(datetime.datetime.now() - t1)

0:00:51.903597


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(vectors, balanced_y, test_size=0.20, random_state=42)

In [4]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn import tree

# initialise the Naive-Bayes classifier
#classifier = GaussianNB()
#classifier = MultinomialNB()
#classifier = ComplementNB()
classifier = LinearSVC()
#classifier = tree.DecisionTreeClassifier()

#Run it
t1 = datetime.datetime.now()
#X_train = X_train.toarray()
classifier.fit(X_train, y_train)
print(datetime.datetime.now() - t1)

0:00:25.530381


In [5]:
preds = classifier.predict(X_test)
print(list(preds[:10]))
print(y_test[:10])

['2\n', '2\n', '5\n', '3\n', '4\n', '1\n', '1\n', '1\n', '2\n', '2\n']
['2\n', '3\n', '5\n', '3\n', '4\n', '1\n', '1\n', '1\n', '3\n', '2\n']


In [6]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, preds))


0.60384


In [7]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

          1
       0.70      0.78      0.74      5057
          2
       0.53      0.49      0.51      5010
          3
       0.52      0.48      0.50      4999
          4
       0.53      0.51      0.52      4954
          5
       0.70      0.76      0.73      4980

    accuracy                           0.60     25000
   macro avg       0.60      0.60      0.60     25000
weighted avg       0.60      0.60      0.60     25000



In [8]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, preds))

[[3930  903  140   31   53]
 [1272 2447 1017  197   77]
 [ 280 1075 2392 1048  204]
 [  70  191  888 2540 1265]
 [  64   36  149  944 3787]]
